In [1]:
from pymongo import MongoClient
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from functools import reduce
import operator
import json

In [2]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {"type": "Point", "coordinates": [data["latt"], data["longt"]]}
    except:
        return data

In [3]:
load_dotenv()
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [6]:
lisboa = geocode("Avenida Antonio Augusto de Aguiar, Lisboa") #coordenadas de Madrid

In [7]:
lisboa

{'type': 'Point', 'coordinates': ['38.73280', '-9.15221']}

In [8]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [9]:
def extraetodo(json):
    todo = {"nombre": ["name"], "latitud": ["location", "lat"], "longitud": ["location", "lng"]} 
    total = []
    for elemento in json:
        place = {key: getFromDict(elemento, value) for key,value in todo.items()}
        place["location"] = type_point([place["latitud"], place["longitud"]])
        total.append(place)
    return total

In [10]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [11]:
def funcion(string):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    url_recomendados = 'https://api.foursquare.com/v2/venues/explore'
    parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{lisboa['coordinates'][0]}, {lisboa['coordinates'][1]}", #aquí pongo la ciudad que quiero
    "query": f"{string}" #aquí pongo lo que quiero buscar en la ciudad.
}
    resp = requests.get(url_query, params = parametros).json()
    data = resp["response"]["venues"]
    df = pd.DataFrame(data)
    map_ = ["location", "lat"]
    getFromDict(resp["response"]["venues"][0], map_)
    resp["response"]["venues"][0]["location"]["address"]
    loquebusco = resp["response"]["venues"]
    lovemosclaro = extraetodo(loquebusco)
    data_final = pd.DataFrame(lovemosclaro)
    #download data as json
    with open ("lisboa.json","w") as f: # creamos un archivo vacío en el que vamos a escribir
        json.dump(lovemosclaro,f) # cargamos nuestra lista de diccionarios en ese archivo
    
    return data_final

# STARBUCKS

In [12]:
funcion("Starbucks")

,nombre,latitud,longitud,location
0,Starbucks,38.733999,-9.154119,"{'type': 'Point', 'coordinates': [38.733998594..."
1,Starbucks,38.714602,-9.140608,"{'type': 'Point', 'coordinates': [38.714601625..."
2,Starbucks,38.768755,-9.095954,"{'type': 'Point', 'coordinates': [38.768755, -..."
3,Starbucks,38.710647,-9.139458,"{'type': 'Point', 'coordinates': [38.710646513..."
4,Starbucks,38.697414,-9.203682,"{'type': 'Point', 'coordinates': [38.697414191..."
5,Starbucks,38.777229,-9.219929,"{'type': 'Point', 'coordinates': [38.777229420..."
6,Starbucks CC Alegro,38.728277,-9.218757,"{'type': 'Point', 'coordinates': [38.728277094..."
7,Starbucks,38.771356,-9.130829,"{'type': 'Point', 'coordinates': [38.771355771..."
8,Starbucks,38.752538,-8.941244,"{'type': 'Point', 'coordinates': [38.752537867..."
9,Starbucks,38.660916,-9.176244,"{'type': 'Point', 'coordinates': [38.660915749..."


#### WE VISUALIZE IT

In [13]:
def visualizar ():
    starbuscks_lisbon = funcion("Starbucks")
    gdf = gpd.GeoDataFrame(starbuscks_lisbon, geometry=gpd.points_from_xy(starbuscks_lisbon.longitud, starbuscks_lisbon.latitud))
    return Map(Layer(gdf, "color:purple", popup_hover=[popup_element("nombre", "Lisbon Starbucks")]))
visualizar()

# Train Station

In [14]:
funcion("Estação Ferroviária") #quiero sólo la 0 y la 1

,nombre,latitud,longitud,location
0,Estação Ferroviária de Entrecampos,38.744562,-9.148288,"{'type': 'Point', 'coordinates': [38.744561584..."
1,Estação Ferroviária do Cais do Sodré,38.706094,-9.145466,"{'type': 'Point', 'coordinates': [38.706094077..."
2,Estação Ferroviária de Sete Rios,38.740383,-9.166768,"{'type': 'Point', 'coordinates': [38.740382659..."
3,Estação Ferroviária do Rossio,38.714373,-9.141132,"{'type': 'Point', 'coordinates': [38.714373031..."
4,Estação Ferroviária de Campolide,38.732494,-9.168048,"{'type': 'Point', 'coordinates': [38.732493734..."
5,Estação Ferroviária Roma-Areeiro,38.745784,-9.135414,"{'type': 'Point', 'coordinates': [38.745783868..."
6,Estação Ferroviária de Santos,38.706264,-9.155231,"{'type': 'Point', 'coordinates': [38.706264208..."
7,Estação Ferroviária de Santa Apolónia,38.713923,-9.122520,"{'type': 'Point', 'coordinates': [38.713922713..."
8,Estação Ferroviária de Alcântara-Mar,38.702345,-9.174232,"{'type': 'Point', 'coordinates': [38.702344835..."
9,Estação Ferroviária de Sintra,38.799415,-9.385602,"{'type': 'Point', 'coordinates': [38.799414750..."


### WE VISUALIZE IT

In [15]:
def visualizar ():
    stations_lisbon = funcion("Estação Ferroviária")
    gdf = gpd.GeoDataFrame(stations_lisbon, geometry=gpd.points_from_xy(stations_lisbon.longitud, stations_lisbon.latitud))
    return Map(Layer(gdf, "color:purple", popup_hover=[popup_element("nombre", "Lisbon Train Stations")]))
visualizar()

# DAY CARE

In [16]:
funcion("infantário") #infantário #jardim de infância

,nombre,latitud,longitud,location
0,Infantário de Nossa Senhora da Purificação,38.746174,-9.142780,"{'type': 'Point', 'coordinates': [38.746173779..."
1,Infantário ReiBebé,38.752801,-9.149980,"{'type': 'Point', 'coordinates': [38.752800604..."
2,Infantario Os Pequenos Indios,38.749365,-9.168598,"{'type': 'Point', 'coordinates': [38.749365308..."
3,Infantário Letrinhas - Faculdade de Letras da ...,38.754434,-9.158515,"{'type': 'Point', 'coordinates': [38.754433636..."
4,Infantário TAP,38.777046,-9.126870,"{'type': 'Point', 'coordinates': [38.777046, -..."
5,Infantario Os 2patinhos,38.655961,-9.163559,"{'type': 'Point', 'coordinates': [38.655961225..."
6,Infantario,38.789281,-9.323794,"{'type': 'Point', 'coordinates': [38.789281, -..."
7,Infantário Peixinho Azul,38.821571,-9.173123,"{'type': 'Point', 'coordinates': [38.821571126..."
8,Infantario o papagaio,38.711104,-8.982382,"{'type': 'Point', 'coordinates': [38.711103534..."
9,Praceta do Infantário,38.747161,-9.230004,"{'type': 'Point', 'coordinates': [38.747161498..."


### WE VISUALIZE IT

In [17]:
def visualizar ():
    lisbon_daycare = funcion("infantário")
    gdf = gpd.GeoDataFrame(lisbon_daycare, geometry=gpd.points_from_xy(lisbon_daycare.longitud, lisbon_daycare.latitud))
    return Map(Layer(gdf, "color:purple", popup_hover=[popup_element("nombre", "Lisbon Day Care")]))
visualizar()